In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
import os, sys
import json

from run_search import find_similar_vectors

In [10]:
sys.path.append('../pubmed_rag')

In [11]:
from model import get_tokens, get_sentence_embeddings

Using this notebook to develop the prompt for Llama3.1. 

Requires that the DB already made.

In [27]:
vector_query = '''
what databases should be used for a knowledge graph that will be used for drug repurposing or target discovery?
'''
path_config = '/users/anglup/GitHub/pubmed-rag/config.yaml'
path_to_repo = '/users/anglup/github/pubmed-rag'

In [28]:
results = find_similar_vectors(
    path_config,
    vector_query,
)
# for x in (results):
#     print(json.dumps(x, indent=4))

Path to config file: /users/anglup/GitHub/pubmed-rag/config.yaml
Loading config params ... 
Configuration: {'pmid file path': 'notebooks/data/test_pmid_list.csv', 'biocjson output path': 'biocjson', 'max_tokens': 500, 'transformer_model': 'NeuML/pubmedbert-base-embeddings', 'output_dimensions': 768, 'tsne': 'tsne-neuml.png', 'db name': '/users/anglup/github/pubmed-rag/demo.db', 'metric_type': 'COSINE', 'n_results': 10}
Accessing /users/anglup/github/pubmed-rag/demo.db
Loading model NeuML/pubmedbert-base-embeddings from HuggingFace
Embedding query: 
what databases should be used for a knowledge graph that will be used for drug repurposing or target discovery?

Searching against vector database


In [29]:
results

[{'id': 167,
  'distance': 0.6122684478759766,
  'entity': {'section': 'results',
   'pmid': 37188275,
   'text': 'However, some highly druggable targets associated with dozens of antischizophrenia drugs, such as dopamine receptor D2 (DRD2) and 5-hydroxytryptamine receptor 1A (HTR1A), were overlooked by iRIGS . It is interesting to notice that both targets (DRD2 and HTR1A) could be successfully predicted by GraphEvo (Table S6). Explainability of ESKG in gene druggability prediction . The above results demonstrated the satisfying performance of the ESKG-derived model (GraphEvo) in the task of gene druggability prediction . We speculated that an important reason for these results is that highly druggable targets may have some evolutionary hallmarks, which could be extracted from the ESKG . It was found that 78.95% of highly druggable targets are Ohnologs (Fig . 3C and Table S5), which are generated by two WGD events in the early vertebrate lineage and are strongly associated with disease

Now, getting Llama loaded in here ..

In [30]:
import requests
import json

url = "http://localhost:11434/api/chat"
model = "llama3.1"

In [34]:
def llama3(prompt, model='llama3.1'):
    data = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False,
    }

    headers = {
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()["message"]["content"]


In [35]:
response = llama3("who are the main characters of friends?")

In [36]:
print(response)

The six main characters in the popular TV show "Friends" are:

1. **Rachel Green** (played by Jennifer Aniston): A spoiled rich girl who leaves her fiancé at the altar and moves to New York City, starting a new life with her friend Monica.
2. **Monica Geller** (played by Courteney Cox): A neat freak, organized, and caring chef who is often the glue that holds the group together.
3. **Ross Geller** (played by David Schwimmer): Monica's older brother, a paleontologist, and a divorced father who has had a long-standing crush on Rachel since high school.
4. **Joey Tribbiani** (played by Matt LeBlanc): A struggling actor and womanizer who is known for his good looks, love of food, and simple-mindedness.
5. **Chandler Bing** (played by Matthew Perry): A sarcastic and witty advertising executive who uses humor to hide his emotional pain and awkward past.
6. **Phoebe Buffay** (played by Lisa Kudrow): A quirky, offbeat, and eccentric masseuse who is known for her weird habits, singing talents, 